XNLI

In [ ]:
from tqdm import tqdm

lang_accuracies = {}
for lang in langs:
    print(f"Processing {lang}...")
    predictions = []
    actual_labels = []
    for batch in tqdm(xnli_dataloaders[lang]):

        tok1 = tokenizer(batch['statement1'], return_tensors='pt', padding=True).to(device)
        tok2 = tokenizer(batch['statement2'], return_tensors='pt', padding=True).to(device)
        tok3 = tokenizer(batch['statement3'], return_tensors='pt', padding=True).to(device)
        labels = batch['label']

        prob1 = F.softmax(model(input_ids=tok1['input_ids'], attention_mask=tok1['attention_mask']).logits, dim=-1)[:,1]
        prob2 = F.softmax(model(input_ids=tok2['input_ids'], attention_mask=tok2['attention_mask']).logits, dim=-1)[:,1]
        prob3 = F.softmax(model(input_ids=tok3['input_ids'], attention_mask=tok3['attention_mask']).logits, dim=-1)[:,1]

         # Stack probabilities into a tensor along the last dimension
        stacked_probs = torch.stack([prob1, prob2, prob3], dim=-1)

        # Find the index of the maximum probability for each example in the batch
        preds = torch.argmax(stacked_probs, dim=-1)
        
        predictions.extend(preds.cpu().tolist())
        actual_labels.extend(labels.cpu().tolist())
    lang_accuracies[lang] = clf_metrics.compute(predictions=predictions, references=actual_labels)['accuracy']

XWinoGrad

In [ ]:
xwinograd_dataloaders = {}
for lang in langs:
    xwinograd_dataloaders[lang] = DataLoader(xwinograd_statements[lang], batch_size=32, shuffle=False)

In [ ]:
from tqdm import tqdm

lang_accuracies = {}

for lang in langs:
    print(f"Processing {lang}...")
    predictions = []
    actual_labels = []
    for batch in tqdm(xwinograd_dataloaders[lang]):

        tok1 = tokenizer(batch['statement1'], return_tensors='pt', padding=True).to(device)
        tok2 = tokenizer(batch['statement2'], return_tensors='pt', padding=True).to(device)

        labels = batch['answer']
        prob1 = F.softmax(model(input_ids=tok1['input_ids'], attention_mask=tok1['attention_mask']).logits, dim=-1)[:,1]
        prob2 = F.softmax(model(input_ids=tok2['input_ids'], attention_mask=tok2['attention_mask']).logits, dim=-1)[:,1]
        
        preds = torch.argmax(torch.stack([prob1, prob2],dim=-1),dim=-1)
        predictions.extend(preds.cpu().tolist())
        actual_labels.extend(labels.cpu().tolist())

    lang_accuracies[lang] = clf_metrics.compute(predictions=predictions, references=actual_labels)['accuracy']

XstoryCloze

In [ ]:
xstorycloze_dataloaders = {}
for lang in langs:
    xstorycloze_dataloaders[lang] = DataLoader(xstorycloze_statements[lang], batch_size=32, shuffle=False)

In [ ]:
from tqdm import tqdm

lang_accuracies = {}

for lang in langs:
    print(f"Processing {lang}...")
    predictions = []
    actual_labels = []
    for batch in tqdm(xwinograd_dataloaders[lang]):

        tok1 = tokenizer(batch['statement1'], return_tensors='pt', padding=True).to(device)
        tok2 = tokenizer(batch['statement2'], return_tensors='pt', padding=True).to(device)

        labels = batch['answer_right_ending']
        prob1 = F.softmax(model(input_ids=tok1['input_ids'], attention_mask=tok1['attention_mask']).logits, dim=-1)[:,1]
        prob2 = F.softmax(model(input_ids=tok2['input_ids'], attention_mask=tok2['attention_mask']).logits, dim=-1)[:,1]
        
        preds = torch.argmax(torch.stack([prob1, prob2],dim=-1),dim=-1)
        predictions.extend(preds.cpu().tolist())
        actual_labels.extend(labels.cpu().tolist())

    lang_accuracies[lang] = clf_metrics.compute(predictions=predictions, references=actual_labels)['accuracy']